In [1]:
import pandas as pd
from pypfopt.expected_returns import returns_from_prices 
from data_capture import FromYF
import inputs
from core.regression import regression
from core.logic import Filtration, Allocation
from core.performance import regressed_expected_returns

In [2]:
watchlist_df = pd.read_excel(inputs.WATCHLIST_PATH,index_col=0)
watchlist = watchlist_df.index.tolist()
watchlist.append(inputs.MARKET)

price_df = FromYF.get_prices(watchlist,period=inputs.PERIOD,interval=inputs.INTERVAL)

returns_df = returns_from_prices(price_df,log_returns=inputs.IS_CONTINOUSLY_COMPOUNDED)

returns_df

[*********************100%***********************]  5 of 5 completed


,AAPL,AMZN,GOOGL,TSLA,^RUA
Date,,,,,
2020-07-28,-0.016428,-0.017963,-0.016856,-0.040991,-0.007094
2020-07-29,0.019168,0.011065,0.013208,0.015320,0.013811
2020-07-30,0.012100,0.006049,0.009754,-0.007751,-0.003390
2020-07-31,0.104689,0.036961,-0.032775,-0.038138,0.005645
2020-08-03,0.025198,-0.016681,-0.003488,0.037910,0.008723
...,...,...,...,...,...
2022-07-20,0.013510,0.038575,0.000791,0.008023,0.007983
2022-07-21,0.015094,0.015150,0.003863,0.097805,0.009424
2022-07-22,-0.008111,-0.017732,-0.056323,0.001975,-0.010572


In [3]:
regression_df = regression(returns_df, inputs.MARKET)

selection_df = Filtration.significance_by_beta(regression_df,inputs.MIN_BETA, inputs.MAX_BETA)

selection_df

,alpha,betav,alpha_t_stat,beta_t_stat,skewness,kurtoses,betaub,betalb,alphalb,alphaub,epsilon
tickers,,,,,,,,,,,
AAPL,0.000615,1.293899,1.010031,24.603101,0.860,9.316,1.190572,1.190572,-0.000581,0.001811,0.335098
AMZN,-0.000905,1.401185,-1.255946,22.525170,0.679,13.416,1.278968,1.278968,-0.002320,0.000510,0.362883


In [4]:
E_df = regressed_expected_returns(returns_df,inputs.MARKET)

E_df = E_df.loc[selection_df.index.values]

E_df

,predict,predict_lower,predict_upper
tickers,,,
AAPL,0.001166,-0.025620,0.027953
AMZN,-0.000308,-0.031991,0.031376


In [5]:
cov_matrix_df = returns_df[selection_df.index.values].cov(min_periods=None, ddof=1)

cov_matrix_df

,AAPL,AMZN
AAPL,0.000410,0.000303
AMZN,0.000303,0.000522


In [6]:
market_returns_s = returns_df[inputs.MARKET]

position_df = Allocation.by_method(E_df,cov_matrix_df,market_returns_s,inputs.ALLOCATION_METHOD)

position_df.to_excel(inputs.POSITION_PATH, sheet_name = str(pd.to_datetime('today').date()))

position_df


,weights
AAPL,0.672433
AMZN,0.327567
